In [ ]:
import math
import logging
import os
import glob
import numpy as np
import pandas as pd
import scipy.signal as sig

import sys, os, os.path

sys.path.append(os.path.expanduser("../src"))

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from spinorama.load_misc import graph_melt
from spinorama.load_rewseq import parse_eq_iir_rews
from spinorama.plot import plot_spinorama, plot_graph
from spinorama.filter_iir import Biquad
from spinorama.filter_peq import peq_build, peq_freq

In [ ]:
nb_points = 200
flat_u = pd.DataFrame(
    {
        "Freq": np.logspace(1.0 + math.log10(2), 4.0 + math.log10(2), nb_points),
        "On Axis": [0] * nb_points,
    }
)
flat = graph_melt(flat_u)

In [ ]:
my_fs = 48000
eq_dirs = glob.glob("../datas/eq/*")
peqs = {}
count = 0
for d in eq_dirs:
    if os.path.isdir(d):
        speaker = os.path.basename(d)
        iirname = "{0}/iir-*.txt".format(d)
        peqs[speaker] = {}
        for iir in glob.glob(iirname):
            if os.path.isfile(iir):
                biir = os.path.basename(iir)
                peqs[speaker][biir] = parse_eq_iir_rews(iir, my_fs)
                count += 1
print("Found {0} IIR eq".format(count))

In [ ]:
peqs["Infinity R152"]

In [ ]:
nb_points = 200
freqs = np.logspace(1.0 + math.log10(2), 4.0 + math.log10(2), nb_points)

In [ ]:
def plot_eq(speaker):
    traces = []
    for name, peq in peqs[speaker].items():
        traces.append(go.Scatter(x=freqs, y=peq_build(freqs, peq), name=name))

    if len(traces) < 2:
        return

    fig = go.Figure(data=traces)
    fig.update_xaxes(
        dict(
            title_text="Frequency (Hz)",
            type="log",
            range=[math.log10(20), math.log10(20000)],
            showline=True,
            dtick="D1",
        )
    )
    fig.update_yaxes(
        dict(
            title_text="SPL (dB)",
            range=[-5, 5],
            showline=True,
            dtick="D1",
        )
    )
    fig.update_layout(title="{}: various EQs ".format(speaker))
    return fig

In [ ]:
for speaker in peqs.keys():
    fig = plot_eq(speaker)
    if fig:
        fig.show()

In [ ]:
plot_eq("NHT SB2")

In [ ]:
plot_eq("Klipsch RP-600M")

In [ ]:
plot_eq("KEF Reference 1 Meta")

In [ ]:
plot_eq("Polk Audio Signature Elite ES20")